In [1]:
import scipy.io as spio
import numpy as np
import vtk
import pdb
import matplotlib.pyplot as plt

In [2]:
covis = spio.loadmat("data/Processed/2021/01/03/COVIS-20210103T000002-fullimaging1/COVIS-20210103T000002-imaging1.mat")

xg = covis["imaging"]["grid"][0][0][0]["x"][0]
yg = covis["imaging"]["grid"][0][0][0]["y"][0]
zg = covis["imaging"]["grid"][0][0][0]["z"][0]
vg = covis["imaging"]["grid"][0][0][0]["Id_filt"][0]

v = vg.copy()
eps = np.nan
m = np.where(v == 0)
v[m] = 1  # remove zeros
v = 10 * np.log10(v)

In [3]:
v.shape

(241, 241, 81)

In [4]:
# Create a VTK structured points object for the second dataset
points2 = vtk.vtkPoints()
scalars2 = vtk.vtkFloatArray()

for i in range(xg.shape[0]):
    for j in range(xg.shape[1]):
        for k in range(xg.shape[2]):
            points2.InsertNextPoint(xg[i, j, k], yg[i, j, k], zg[i, j, k])
            scalars2.InsertNextValue(v[i, j, k])

vtk_data2 = vtk.vtkStructuredPoints()
vtk_data2.SetDimensions(xg.shape[2], xg.shape[1], xg.shape[0])
vtk_data2.GetPointData().SetScalars(scalars2)

vtk_data2.SetExtent(0, xg.shape[2]-1, -xg.shape[1]+1, 0, 0, xg.shape[0]-1)
vtk_data2.SetSpacing(0.25,0.25,0.25)
vtk_data2.SetOrigin(-2,20, -40)

In [42]:
# Create a VTK contour filter
contour2 = vtk.vtkContourFilter()
contour2.SetInputData(vtk_data2)
contour2.SetValue(0, (np.nanmax(v) + np.nanmin(v))/2)

In [8]:
writer = vtk.vtkXMLImageDataWriter()
writer.SetFileName("output.vti")
writer.SetInputData(vtk_data2)
writer.Write()

1